In [8]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env')

import requests
import os
from datetime import datetime, timedelta
from pprint import pprint
import pandas as pd
from pathlib import Path

TODO
- Perform Initial Load from API into a database
- Perform Incremental Load from API into a database

Initial Load
- How to fetch using the url? - Work in progress
- Where to dump the data initially? (S3) - Yes
- Do you need to dump the data? - Yes: Avoid re-fetching from API if pipeline fails

In [2]:
(datetime.now() - timedelta(days=13)).isoformat()[:-3]

'2025-07-08T14:33:34.627'

In [23]:
APP_TOKEN = os.getenv('APP_TOKEN')

# Assume last_update was 7 days ago
last_update = (datetime.now() - timedelta(days=13)).isoformat()[:-3]

# For Backfill
min_update = f"SELECT min(updated_on)"
max_update = f"SELECT max(updated_on)"

# Implementing CDC, batch incremental loads only
query = f"SELECT * WHERE updated_on >= '{last_update}'"
null_query = f"SELECT * WHERE updated_on = NULL"

url = "https://data.cityofchicago.org/api/v3/views/crimes/query.json"
headers = {
    'X-App-Token' : APP_TOKEN
}

# If status code != 200, stop
# If response is empty, stop
# Failsafe, if response doesnt return non-empty, stop pagination after certain page number
results = []
pagenum = 1
pagesize = 50000
while True:
    print(pagenum)
    body = {
        'query' : query,
        'page' : {
            'pageNumber' : pagenum,
            'pageSize' : pagesize
        },
        "includeSynthetic": False
    }

    res = requests.post(
        url,
        json=body,
        headers=headers
    )

    if res.status_code != 200:
        break

    if res.json() == []:
        break
    
    if pagenum >= 15:
        break
    
    results.extend(res.json())
    pagenum += 1

1
2


In [ ]:
df = pd.json_normalize(results)

save_path = ''
filename = ''
df.to_json("./data.json.gz", compression='gzip', orient='records')

In [ ]:
TMP_FOLDER = Path('./temp')